In [17]:
import dwavebinarycsp
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector


# Defining the logic circuit

In [23]:
def xor_circuit(a,b, z):
    return (((not a) and b) or ((not b) and a))==z

# Constraint satisfaction problem
csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)

csp.add_constraint(xor_circuit, ['a', 'b', 'z'])
print("----------")

print("Logic table")
print("a b z  V?")
for a in [0,1]:
    for b in [0, 1]:
        for z in [0,1]:
            print(f"{a} {b} {z} {csp.check({'a':a, 'b':b, 'z':z})}")

print("----------")
print("Computing Binary Quadratic Model")
# Binary Quadratic Model
bqm = dwavebinarycsp.stitch(csp)
# https://github.com/dwavesystems/dwavebinarycsp/blob/master/dwavebinarycsp/compilers/stitcher.py
print("Finished stitching")

print(bqm)

----------
Logic table
a b z  V?
0 0 0 True
0 0 1 False
0 1 0 False
0 1 1 True
1 0 0 False
1 0 1 True
1 1 0 True
1 1 1 False
----------
Computing Binary Quadratic Model
Finished stitching
BinaryQuadraticModel({'a': 2.0, 'aux0': 4.0, 'aux1': 4.0, 'b': 2.0, 'z': 2.0}, {('aux0', 'a'): -4.0, ('aux1', 'a'): -4.0, ('aux1', 'aux0'): 0.0, ('b', 'a'): 4.0, ('b', 'aux0'): -4.0, ('b', 'aux1'): -4.0, ('z', 'a'): -4.0, ('z', 'aux0'): 4.0, ('z', 'aux1'): 4.0, ('z', 'b'): -4.0}, 0.0, 'BINARY')


In [24]:
# Set up a D-Wave system as the sampler
sampler = EmbeddingComposite(DWaveSampler())

print()
sampleset = sampler.sample(bqm, num_reads=100, label="XOR test")

# Check how many solutions meet the constraints (are valid)
print("Checking solutions")
valid, invalid, data = 0, 0, []
for datum in sampleset.data(['sample', 'energy', 'num_occurrences']):
    # print(datum.sample)
    if (csp.check(datum.sample)):
        valid = valid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '1'))
    else:
        invalid = invalid+datum.num_occurrences
        #print(f"Failed: {datum.sample} {datum.energy}")
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '0'))

print(valid, invalid)        


Checking solutions
88 12


In [26]:
sampleset.info

{'timing': {'qpu_sampling_time': 13366.0,
  'qpu_anneal_time_per_sample': 20.0,
  'qpu_readout_time_per_sample': 93.12,
  'qpu_access_time': 21813.4,
  'qpu_access_overhead_time': 9519.6,
  'qpu_programming_time': 8447.4,
  'qpu_delay_time_per_sample': 20.54,
  'post_processing_overhead_time': 308.0,
  'total_post_processing_time': 308.0},
 'problem_id': '310336b3-85cd-41ad-aff7-db2bef892ab4',
 'problem_label': 'XOR test',
 'embedding_context': {'embedding': {'aux0': (943,),
   'a': (5359,),
   'aux1': (958,),
   'b': (5344,),
   'z': (5285, 1017)},
  'chain_break_method': 'majority_vote',
  'embedding_parameters': {},
  'chain_strength': 2.6828763668868527},
 'warnings': [{'type': dwave.system.warnings.ChainStrengthWarning,
   'message': 'Some quadratic biases are stronger than the given chain strength',
   'level': 30,
   'data': {'source_interactions': [('aux0', 'a'),
     ('aux1', 'a'),
     ('b', 'a'),
     ('b', 'aux0'),
     ('b', 'aux1'),
     ('z', 'a'),
     ('z', 'aux0'),
  

In [27]:
dwave.inspector.show(sampleset)

# Weak chain strength?

Opening in existing browser session.


'http://127.0.0.1:18002/?problemId=310336b3-85cd-41ad-aff7-db2bef892ab4'